In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_diagonal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,10825.41,-0.04,0.00,-0.02,-0.01,-0.00,-0.00,0.00
3,-0.04,2292.40,-0.00,0.01,0.00,0.00,0.00,-0.00
4,0.00,-0.00,408.88,-0.01,-0.00,0.00,-0.00,0.00
5,-0.02,0.01,-0.01,67.80,0.00,0.00,0.00,-0.00
6,-0.01,0.00,-0.00,0.00,10.54,-0.00,0.00,0.00
7,-0.00,0.00,0.00,0.00,-0.00,1.61,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.31,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.05


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00000,-0.00002,-0.00003,-0.00001,-0.00002,0.00000
3,-0.00001,1.00000,-0.00000,0.00002,0.00001,0.00002,0.00000,-0.00001
4,0.00000,-0.00000,1.00000,-0.00004,-0.00002,0.00004,-0.00001,0.00001
5,-0.00002,0.00002,-0.00004,1.00000,0.00002,0.00000,0.00001,-0.00002
6,-0.00003,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00002,0.00003
7,-0.00001,0.00002,0.00004,0.00000,-0.00001,1.00000,-0.00001,-0.00001
8,-0.00002,0.00000,-0.00001,0.00001,0.00002,-0.00001,1.00000,0.00001
9,0.00000,-0.00001,0.00001,-0.00002,0.00003,-0.00001,0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.06179789513666569

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.46631802e+07 6.43475317e+05 1.99071529e+04 5.25497510e+02
 1.19197274e+01 2.48860582e-01 6.96858542e-03 5.61887858e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999485,-0.031621,-0.005359,-0.000889,-0.000139,-0.000021,-0.000004,-6.098667e-07
1,0.031436,0.999017,-0.030846,-0.004925,-0.000765,-0.000116,-0.000023,-3.505231e-06
2,0.006288,0.030472,0.998916,-0.034230,-0.005151,-0.000783,-0.000152,-2.303668e-05
3,0.001250,0.005897,0.033820,0.998593,-0.039933,-0.005922,-0.001139,-1.749630e-04
4,0.000243,0.001142,0.006415,0.039355,0.997790,-0.052208,-0.009747,-1.488351e-03
5,0.000049,0.000229,0.001284,0.007759,0.050687,0.993650,-0.099111,-1.434894e-02
6,0.000014,0.000065,0.000368,0.002217,0.014266,0.095136,0.983474,-1.533596e-01
7,0.000004,0.000020,0.000115,0.000692,0.004446,0.029117,0.151193,9.880651e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005148233487990428,
 0.0009827542496454456,
 0.001084055001524309,
 0.0014065656363380086,
 0.002209542173744694,
 0.006350178503526704,
 0.01652589179817132,
 0.01193487695240636]